# Chatbot QA sur document PDF avec LangChain & Hugging Face


Ce notebook présente un use case de QA avec conversation (donc chatbot) traité avec Langchain.

Ce notebook fait suite à celui sur le QA. Il sera beaucoup plus détaillé que celui-ci car il en reprend une grande partie.

Contrairement au notebook QA, ici on souhaite pouvoir poursuivre la conversation avec le chatbot. Il faut donc prendre en compte l'historique de la conversation. Cependant, à chaque question posée, la chaîne effectue une recherche par similarité.

Sources


*   https://python.langchain.com/docs/modules/chains/popular/vector_db_qa.html
*   https://github.com/gkamradt/langchain-tutorials/blob/main/LangChain%20Cookbook%20Part%202%20-%20Use%20Cases.ipynb

## import

In [1]:
!pip install -qU langchain tiktoken chromadb pypdf transformers InstructorEmbedding sentence_transformers
!pip install -qU accelerate bitsandbytes sentencepiece Xformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 54.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.3/399.3 kB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.0/255.0 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 71.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 4.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

## Import du modèle




In [2]:
import torch
import transformers
from transformers import GenerationConfig, pipeline, AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bigscience/bloomz-560m") #3b 1b1 560m
model = AutoModelForCausalLM.from_pretrained("bigscience/bloomz-560m")#,load_in_8bit=True, device_map='auto', torch_dtype=torch.float16, low_cpu_mem_usage=True)

In [3]:
from transformers import pipeline
from langchain.llms import HuggingFacePipeline
import torch

pipe = pipeline(
    #"conversational",
    "text-generation",
    model=model,
    tokenizer=tokenizer,
   # min_length=10,
    max_length=1024,
    temperature=0,
    top_p=0.95,
    repetition_penalty=1.15
)

local_llm = HuggingFacePipeline(pipeline=pipe)

Quelques questions d'échauffement.

In [5]:
print(local_llm('What is the capital of France ?'))

 Paris


In [6]:
print(local_llm('Which day comes after friday ?'))

 Saturday


In [7]:
print(local_llm('What is the color of the sky ?'))

 blue


# LangChain multi-doc retriever with ChromaDB

***New Points***
- Multiple Files - PDFs
- ChromaDB : Chroma is the open-source embedding database. Chroma makes it easy to build LLM apps by making knowledge, facts, and skills pluggable for LLMs.
- Local LLM
- Instuctor Embeddings


## Setting up LangChain


In [8]:
import os

In [9]:
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader

from langchain.embeddings import HuggingFaceInstructEmbeddings

## Load multiple and process documents

In [10]:
# Load and process the text files

# Pour un seul fichier
# loader = TextLoader('attention.txt')

loader = DirectoryLoader('./documents/', glob="./*.pdf", loader_cls=PyPDFLoader)

documents = loader.load()

In [11]:
len(documents)

40

In [12]:
documents[0]

Document(page_content="1 janvier2015\nC O N S T I T U T I O N\nLe Gouvernement de la République, conformément\nàla loi constitutionnelledu 3juin 1958,a proposé,\nLepeuplefrançais aadopté,\nLe Président de la République promulgue la loi\nconstitutionnelledontlateneursuit:\nPRÉAMBULE\nLe peuple français proclame solennellement son attachement aux Droits de\nl'homme et aux principes delasouveraineté nationaletels qu'ils ont été définis parla\nDéclaration de 1789, confirmée et complétée par le préambule de la Constitution de\n1946, ainsi qu'aux droits et devoirs définis dans la Charte de l'environnement de\n2004.\nEn vertu de ces principes et de celui de la libre détermination des peuples, la\nRépublique offre aux territoires d'outre-mer qui manifestent la volonté d'y adhérer\ndes institutions nouvelles fondées sur l'idéal commun de liberté, d'égalité et de\nfraternitéet conçuesen vuedeleurévolutiondémocratique.\n________\nARTICLE PREMIER . La France est une République indivisible, laïque,

In [13]:
#splitting the text into (CharacterTextSplitter)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, # controls the max size (in terms of number of characters) of the final documents
                                               chunk_overlap=200 # specifies how much overlap there should be between chunks.
                                              )
texts = text_splitter.split_documents(documents)

In [14]:
len(texts)

118

In [15]:
texts[0]

Document(page_content="1 janvier2015\nC O N S T I T U T I O N\nLe Gouvernement de la République, conformément\nàla loi constitutionnelledu 3juin 1958,a proposé,\nLepeuplefrançais aadopté,\nLe Président de la République promulgue la loi\nconstitutionnelledontlateneursuit:\nPRÉAMBULE\nLe peuple français proclame solennellement son attachement aux Droits de\nl'homme et aux principes delasouveraineté nationaletels qu'ils ont été définis parla\nDéclaration de 1789, confirmée et complétée par le préambule de la Constitution de\n1946, ainsi qu'aux droits et devoirs définis dans la Charte de l'environnement de\n2004.\nEn vertu de ces principes et de celui de la libre détermination des peuples, la\nRépublique offre aux territoires d'outre-mer qui manifestent la volonté d'y adhérer\ndes institutions nouvelles fondées sur l'idéal commun de liberté, d'égalité et de\nfraternitéet conçuesen vuedeleurévolutiondémocratique.\n________\nARTICLE PREMIER . La France est une République indivisible, laïque,

## HF Instructor Embeddings

In [16]:
from langchain.embeddings import HuggingFaceInstructEmbeddings

instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="ggrn/e5-small-v2", #hkunlp/instructor-xl
                                                      model_kwargs={"device": "cuda"})

load INSTRUCTOR_Transformer
max_seq_length  512


## create the DB

In [17]:
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk
persist_directory = 'db'

## Here is the nmew embeddings being used
embedding = instructor_embeddings

vectordb = Chroma.from_documents(documents=texts,
                                 embedding=embedding,
                                 persist_directory=persist_directory)

## Make a chain with memory
C'est ici le point clé. Pour construire la chaîne, on n'utilise pas `RetrievalQA.from_chain_type` mais `ConversationalRetrievalChain.from_llm`.

Construction de `memory` avec `ConversationBufferMemory`. On peut choisir aussi son propre prompt.

In [78]:
from langchain.memory import ConversationBufferMemory
from langchain import PromptTemplate
template = """You are a AI having a conversation with a human.

  {chat_history}
  Human: {question}
  AI:"""

prompt = PromptTemplate(
      input_variables=["chat_history", "question"], template=template
)

memory = ConversationBufferMemory(memory_key="chat_history", input_key='question', output_key='answer', return_messages=True)

On peut maintenant créer la nouvelle chaîne.

In [79]:
from langchain.chains import ConversationalRetrievalChain

qa = ConversationalRetrievalChain.from_llm(
    local_llm,
    retriever=vectordb.as_retriever(search_kwargs={"k": 3}),
    memory=memory,
    return_source_documents=True,
   # combine_docs_chain_kwargs={"prompt": prompt},
    verbose=True
)

On peut accéder à la mémoire (pour l'instant vide).

In [80]:
qa.memory.chat_memory.messages

[]

Inférence. Avec `verbose=True`, on peut observer le prompt envoyé au LLM. La chaîne renvoie la réponse, mais aussi ses sources.

In [81]:
qa({"question": "Combien de temps est élu le président ?"})



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

conditions énoncées au premier alinéa demeurent réunies. Il se prononce dans les
délais les plus brefs par un avis public. Il procède de plein droit à cet examen et se
prononce dans les mêmes conditions au terme de soixante jours d'exercice des
pouvoirs exceptionnels et à tout moment au-delàdecette durée.
ARTICLE 17. Le Président de la République a le droit de faire grâce à titre
individuel.
ARTICLE 18. Le Président de la République communique avec les deux
assemblées du Parlement par des messages qu'il fait lire et qui ne donnent lieu à
aucundébat.
6

En cas de vacance de la Présidence de la République pour quelque cause que ce
soit, ou d'empêchement constaté par le Conseil constitutionnel saisi par le
Gouvernement et s

{'question': 'Combien de temps est élu le président ?',
 'chat_history': [HumanMessage(content='Combien de temps est élu le président ?', additional_kwargs={}, example=False),
  AIMessage(content=' Cinq', additional_kwargs={}, example=False)],
 'answer': ' Cinq',
 'source_documents': [Document(page_content="conditions énoncées au premier alinéa demeurent réunies. Il se prononce dans les\ndélais les plus brefs par un avis public. Il procède de plein droit à cet examen et se\nprononce dans les mêmes conditions au terme de soixante jours d'exercice des\npouvoirs exceptionnels et à tout moment au-delàdecette durée.\nARTICLE 17. Le Président de la République a le droit de faire grâce à titre\nindividuel.\nARTICLE 18. Le Président de la République communique avec les deux\nassemblées du Parlement par des messages qu'il fait lire et qui ne donnent lieu à\naucundébat.\n6", metadata={'page': 5, 'source': 'documents/constitution.pdf'}),
  Document(page_content="En cas de vacance de la Présidence

Je repose une question. L'historique de la conversation apparait.

In [82]:
qa({"question": "Et pour le sénat ?"})



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: Combien de temps est élu le président ?
Assistant:  Cinq
Follow Up Input: Et pour le sénat ?
Standalone question:

> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

- les compétences de cette collectivité ; sous réserve de celles déjà exercées
par elle, le transfert de compétences de l'État ne peut porter sur les matières
énumérées au quatrième alinéade l'article73, précisées et complétées, lecas
échéant, parlaloi organique;
- les règles d'organisation et de fonctionnement des institutions de la
collectivitéetlerégimeélectoral 

{'question': 'Et pour le sénat ?',
 'chat_history': [HumanMessage(content='Combien de temps est élu le président ?', additional_kwargs={}, example=False),
  AIMessage(content=' Cinq', additional_kwargs={}, example=False),
  HumanMessage(content='Et pour le sénat ?', additional_kwargs={}, example=False),
  AIMessage(content=' 2', additional_kwargs={}, example=False)],
 'answer': ' 2',
 'source_documents': [Document(page_content="- les compétences de cette collectivité ; sous réserve de celles déjà exercées\npar elle, le transfert de compétences de l'État ne peut porter sur les matières\nénumérées au quatrième alinéade l'article73, précisées et complétées, lecas\néchéant, parlaloi organique;\n- les règles d'organisation et de fonctionnement des institutions de la\ncollectivitéetlerégimeélectoral deson assembléedélibérante;\n- les conditions dans lesquelles ses institutions sont consultées sur les projets\net propositions de loi et les projets d'ordonnance ou de décret comportant\ndes dis

In [83]:
# Afficher l'historique
qa.memory.chat_memory.messages

[HumanMessage(content='Combien de temps est élu le président ?', additional_kwargs={}, example=False),
 AIMessage(content=' Cinq', additional_kwargs={}, example=False),
 HumanMessage(content='Et pour le sénat ?', additional_kwargs={}, example=False),
 AIMessage(content=' 2', additional_kwargs={}, example=False)]